In [1]:
import arrow
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt

In [2]:
def map():
  # create space for drawing
  proj = ccrs.PlateCarree()  #create coordinate
  fig = plt.figure(figsize=(10, 5))  #create figure
  ax = fig.subplots(1, 1, subplot_kw={'projection': proj})  #create sub-plot
  # set map parameters
  ax.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8)  #add borders
  ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6)  #add coastlines
  ax.add_feature(cfeat.RIVERS.with_scale('50m'))  #add rivers
  ax.add_feature(cfeat.LAKES.with_scale('50m'))  #add lakes
  # set up gridlines
  gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                   linewidth=1.2, color='k', alpha=0.5, linestyle='--')
  gl.xlabels_top = False  #remove x labels
  gl.ylabels_right = False  #remove y lables
  gl.xformatter = LONGITUDE_FORMATTER  #set x as lon
  gl.yformatter = LATITUDE_FORMATTER  #set y as lat
  return fig, ax

In [3]:
import warnings
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd


warnings.filterwarnings('ignore')


xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

#https://nbviewer.jupyter.org/github/pangeo-data/pangeo-cmip6-examples/blob/master/search_and_load_with_esgf_opendap.ipynb


## Search using ESGF API

In [6]:
#!/usr/bin/env python
# Code from Robinson
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = [] 
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

## Load data with xrray

In [7]:
# Code from Robinson, modified by Yanlei
def CMIP_processing_plot(my_result, my_experiment_id, my_varname, my_model, my_source, index_initial, index_end):
   # there are mulitple sources of the same data--need to pick one
    index_ini=index_initial#3
    index_fin=index_end#11

    files_to_open = my_result[index_ini:index_fin+1]

    n_files= len(files_to_open)
    print ('number of files to open:', n_files)
    
    
    myvarvals=[]
    myvarvals_values = []
    time_appropriate = []
    
    #define a boundary
    ds=xr.open_dataset(files_to_open[0])
    lat = ds.lat
    lon = ds.lon
    lat_range = lat[(lat>=-20)&(lat<=10)]
    lon_range = lon[(lon>=270)&(lon<=330)]
    for ifls in range(n_files):
        ds=xr.open_dataset(files_to_open[ifls])
        # fix CF non standard time issues
        if ds.time.dtype == 'O' and int(ds.indexes['time'][-1].strftime("%Y")) < 2262:
                datetimeindex = ds.indexes['time'].to_datetimeindex()
                ds['time'] = datetimeindex
        # filter some files have different time range
        if ds['time.year'][0].values >= 2065 and ds['time.year'][0].values <= 2099:
            ds_am= ds.sel(lon = lon_range, lat = lat_range)
            myvarvals.append(ds_am)
#             myvarvals_values.append(ds_am.values) 
#             time_appropriate.append(ifls)
#     print(myvarvals)
   # Yanlei
   # concatenate all the data together by dimension "time.year"
    ds_concat = xr.concat(myvarvals, dim="time")
    ds = ds_concat.sel(time=slice('2070-01','2099-12'))
    print(ds.time)

    #     #===================================================
#     #plotting
#     #====================================================


#     DIRin='/Users/yanlei/Documents/PhD/4B/Deep convections in Amazon/future_CAPE_rho_p/2070_2100/ps/' 

#     filename=DIRin+my_experiment_id+'_'+my_varname+'_avg_2070_2100_'+my_model+'_'+my_source+'.nc'
    
#     # save xarrays to netcdf files
#     avgda_ds.to_netcdf(filename)
# #     stdda.to_netcdf(filename)


#     # create map
#     fig, ax = map()
    
#     title= my_experiment_id+'_'+my_varname+'_avg_2070_2100_'+my_model+'_'+my_source
#     avgda_ds.ps.plot.contourf(ax=ax, cmap='Spectral_r', extend='both',
#         transform=ccrs.PlateCarree())
#     plt.title(title)
#     fig.show()

 

## search all models which have available 6hrLev ps data

In [9]:
result_0 = esgf_search(activity_id='ScenarioMIP', table_id='6hrLev', variable_id='ps', experiment_id='ssp585', 
                     latest=True)
#result
for ires in range(len(result_0)):
    print(ires,':', result_0[ires])

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=30
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=390
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=400
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=410
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=420
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&pr

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=770
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=780
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=790
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=800
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&pr

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1150
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1160
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1170
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1180
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=Tru

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1530
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1540
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1550
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=1560
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&latest=Tru

574 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/ssp585/r1i1p1f2/6hrLev/ps/gn/v20200318/ps_6hrLev_MIROC-ES2L_ssp585_r1i1p1f2_gn_203801010600-203901010000.nc
575 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/ssp585/r1i1p1f2/6hrLev/ps/gn/v20200318/ps_6hrLev_MIROC-ES2L_ssp585_r1i1p1f2_gn_203901010600-204001010000.nc
576 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/ssp585/r1i1p1f2/6hrLev/ps/gn/v20200318/ps_6hrLev_MIROC-ES2L_ssp585_r1i1p1f2_gn_204001010600-204101010000.nc
577 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/ssp585/r1i1p1f2/6hrLev/ps/gn/v20200318/ps_6hrLev_MIROC-ES2L_ssp585_r1i1p1f2_gn_204101010600-204201010000.nc
578 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/ssp585/r1i1p1f2/6hrLev/ps/gn/v20200318/ps_6hrLev_MIROC-ES2L_ssp585_r1i1p1f2_gn_2042

1074 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_205106010600-205107010000.nc
1075 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_205107010600-205108010000.nc
1076 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_205108010600-205109010000.nc
1077 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_205109010600-205110010000.nc
1078 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_205110010600-205111010000.nc
1079 : htt

1407 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_207903010600-207904010000.nc
1408 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_207904010600-207905010000.nc
1409 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_207905010600-207906010000.nc
1410 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_207906010600-207907010000.nc
1411 : http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/MIROC/MIROC6/ssp585/r1i1p1f1/6hrLev/ps/gn/v20191114/ps_6hrLev_MIROC6_ssp585_r1i1p1f1_gn_207907010600-207908010000.nc
1412 : htt

## 1. BCC, BCC-CSM2-MR

In [24]:
result_BCC = esgf_search(activity_id='ScenarioMIP', table_id='6hrLev', variable_id='ps', experiment_id='ssp585', 
                    institution_id="BCC", source_id= "BCC-CSM2-MR",latest=True)
#result
for ires in range(len(result_BCC)):
    print(ires,':', result_BCC[ires])

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&institution_id=BCC&source_id=BCC-CSM2-MR&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=6hrLev&variable_id=ps&experiment_id=ssp585&institution_id=BCC&source_id=BCC-CSM2-MR&latest=True&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
0 : http://cmip.bcc.cma.cn/thredds/dodsC/cmip6_data/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/6hrLev/ps/gn/v20190318/ps_6hrLev_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501010000-201912311800.nc
1 : http://cmip.bcc.cma.cn/thredds/dodsC/cmip6_data/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/6hrLev/ps/gn/v20190318/ps_6hrLev_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_202001010000-202412311800.nc
2 : http://cmip.bcc.cma.cn/thredds/dodsC/cmip6_data/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/6hrLev/ps/gn/v2

In [ ]:
fig = CMIP_processing_plot(result_BCC, 'ssp585', 'ps', "BCC", "BCC-CSM2-MR", 10, len(result_BCC))

number of files to open: 8


# 2. CCCma, CanESM5

# 3. CAS, FGOALS-g3